# Notebook for training the models

This model contains all the information required to train all four different models. This uses two main python files: 
- `utils_2D`: Contains all the utility functions required for training the models
- `Kits2019_2D`: Contains the class for the dataset (**Kits20192DDataset**)

In [1]:
!pip install torch==2.3.0 torchaudio==2.3.0 torchvision==0.18.0
!pip install albumentations numpy pandas scikit_learn kaggle
!pip install resnest geffnet opencv-python pretrainedmodels tqdm Pillow packaging monai segmentation_models_pytorch

Training the U-nets

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
from tqdm import tqdm

import segmentation_models_pytorch as smp
import monai.networks.nets as monai_nets

import utils_2D as u
from Kits2019_2D import Kits20192DDataset


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!unzip /content/drive/MyDrive/flattened_data.zip -d flattened_data

Archive:  /content/drive/MyDrive/flattened_data.zip
replace flattened_data/flattened_data/images/image_000000.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Training

In [5]:
def train_model(model, train_loader, val_loader, config):
    """
    Train a MONAI model for medical image segmentation.

    Parameters
    ----------
    model : torch.nn.Module
        The MONAI model to train (e.g., UNet, AttentionUNet, SwinUNETR).
    train_loader : DataLoader
        DataLoader for the training dataset.
    val_loader : DataLoader
        DataLoader for the validation dataset.
    config : dict
        Configuration dictionary containing:
        - "device" (str): Device to train on ("cuda" or "cpu").
        - "lr" (float): Learning rate.
        - "epochs" (int): Number of training epochs.
        - "checkpoint_path" (str): Path to save the best model checkpoint.
        - "num_classes" (int): Number of segmentation classes.

    Returns
    -------
    torch.nn.Module
        The trained model.
    """
    device = config["device"]

    # Wrap model with DataParallel if multiple GPUs are available
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        model = nn.DataParallel(model)

    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])
    criterion = nn.CrossEntropyLoss()

    best_loss = float('inf')
    checkpoint_path = config["checkpoint_path"]

    for epoch in range(config["epochs"]):
        model.train()
        epoch_loss = 0
        for images, masks in train_loader:
            images = images.to(device)
            masks = masks.to(device).long().squeeze(1)  # Fixed typo in squeeze

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{config['epochs']} - Training Loss: {avg_loss:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(device)
                masks = masks.to(device).long().squeeze(1)
                outputs = model(images)
                loss = criterion(outputs, masks)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        print(f"Epoch {epoch + 1}/{config['epochs']} - Validation Loss: {val_loss:.4f}")

        # Save the model with the best validation loss
        if val_loss < best_loss:
            best_loss = val_loss
            # Save the model state dict (handle DataParallel wrapper)
            if isinstance(model, nn.DataParallel):
                torch.save(model.module.state_dict(), checkpoint_path)
            else:
                torch.save(model.state_dict(), checkpoint_path)
            print(f"Model saved with validation loss: {best_loss:.4f}")

    print("Training complete.")
    return model


In [6]:
def create_dataloaders(train_dataset, val_dataset, test_dataset, config):
    """
    Create DataLoaders with multi-GPU support
    """
    batch_size = config["batch_size"]
    num_workers = 4 * torch.cuda.device_count() if torch.cuda.is_available() else 4  # Scale workers with GPUs

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True  # Enables faster data transfer to GPU
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=1,
        pin_memory=True
    )

    return train_loader, val_loader, test_loader


In [7]:
def select_model(model_name, config):
    """
    Initialize and return the segmentation model.
    """
    if model_name == "UNet":
        model = smp.Unet(
            encoder_name="resnet34",
            encoder_weights="imagenet",
            in_channels=config["in_channels"],
            classes=config["num_classes"]
        )
    elif model_name == "UNetV2":
        model = monai_nets.UNet(
            spatial_dims=config["spatial_dims"],
            in_channels=config["in_channels"],
            out_channels=config["out_channels"],
            channels=config["channels"],
            strides=config["strides"]
        )
    elif model_name == "AttentionUNet":
        model = monai_nets.AttentionUnet(
        spatial_dims=config["spatial_dims"],
        in_channels=config["in_channels"],
        out_channels=config["out_channels"],
        channels=config["channels"],
        strides=config["strides"]
    )
    elif model_name == "SwinUNETR":
        model = monai_nets.SwinUNETR(
        spatial_dims=config["spatial_dims"],
        in_channels=config["in_channels"],
        out_channels=config["out_channels"],
        img_size=config["img_size"],
        feature_size=config["feature_size"],
    )
    else:
        raise ValueError(f"Unknown model name: {model_name}")

    return model

In [8]:
def check_mask_values(dataset):
    """Check unique values in masks"""
    all_unique = set()
    for i in range(len(dataset)):
        image, mask = dataset[i]
        unique_values = torch.unique(mask).cpu().numpy()
        all_unique.update(unique_values)
    print(f"All unique values in masks: {sorted(all_unique)}")
    return all_unique

In [9]:
selected_model_name = "SwinUNETR"
config = {
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "lr": 1e-4,
    "epochs": 20,
    "checkpoint_path": f"best_{selected_model_name}.pth",
    "num_classes": 3,
    "in_channels": 1,
    "batch_size": 16 * torch.cuda.device_count() if torch.cuda.is_available() else 16,
    # Add these parameters
    "image_dir": "flattened_data/images",  # Update this path to your image directory
    "mask_dir": "flattened_data/masks",    # Update this path to your mask directory
    "split_train": 0.7,
    "split_val": 0.15,
    "split_test": 0.15,
    "image_size": 256,
    "feature_size": 48,
    "spatial_dims": 2,
    "img_size": (256, 256),
    "out_channels": 3,
    "channels":(16, 32, 64, 128, 256),
    "strides":(2, 2, 2, 2)
}

config["image_dir"] = "/content/flattened_data/flattened_data/images"
config["mask_dir"] = "/content/flattened_data/flattened_data/masks"

In [10]:
image_paths, mask_paths = u.build_dataset_paths(
    images_dir=config["image_dir"],
    masks_dir=config["mask_dir"],
    image_ext=".npy",  # Update if using different extension
    mask_ext=".npy"    # Update if using different extension
)
print(f"Found {len(image_paths)} images and {len(mask_paths)} masks.")
print(f"First image path: {image_paths[0]}")
print(f"First mask path: {mask_paths[0]}")
# Update config with the paths
config["image_paths"] = image_paths
config["mask_paths"] = mask_paths


# files.download(f"best_{selected_model_name}.pth")

Found 16336 images and 16336 masks.
First image path: /content/flattened_data/flattened_data/images/image_000000.npy
First mask path: /content/flattened_data/flattened_data/masks/mask_000000.npy


In [11]:
# Continue with the rest of your code
train_dataset, val_dataset, test_dataset = u.prepare_datasets(config)


# Assuming train_dataset, val_dataset, and test_dataset are already defined
train_loader, val_loader, test_loader = create_dataloaders(train_dataset, val_dataset, test_dataset, config)


# print("Checking mask values...")
# unique_train = check_mask_values(train_dataset)
# unique_val = check_mask_values(val_dataset)
# unique_test = check_mask_values(test_dataset)

In [12]:

# Select and initialize the model

model = select_model(selected_model_name, config)

# Train the selected model
trained_model = train_model(model, train_loader, val_loader, config)

#If in Google Colab, download the trained model

/usr/local/lib/python3.10/dist-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
  warn_deprecated(argname, msg, warning_category)
/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicab

Epoch 1/20 - Training Loss: 0.1093
Epoch 1/20 - Validation Loss: 0.0421
Model saved with validation loss: 0.0421


/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)


Epoch 2/20 - Training Loss: 0.0322
Epoch 2/20 - Validation Loss: 0.0218
Model saved with validation loss: 0.0218


/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)


...

Epoch 18/20 - Training Loss: 0.0067
Epoch 18/20 - Validation Loss: 0.0053
Model saved with validation loss: 0.0053


/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)


Epoch 19/20 - Training Loss: 0.0068
Epoch 19/20 - Validation Loss: 0.0077


/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/albucore/decorators.py:43: UserWarning: HueSaturationValue: hue_shift and sat_shift are not applicable to grayscale image. Set them to 0 or use RGB image
  result = func(img, *args, **kwargs)


Epoch 20/20 - Training Loss: 0.0066
Epoch 20/20 - Validation Loss: 0.0054
Training complete.


In [13]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import segmentation_models_pytorch as smp
from albumentations import Compose, Normalize, Resize
from albumentations.pytorch import ToTensorV2
import os

def calculate_metrics(prediction, ground_truth, num_classes=3):
    """Calculate DSC and IoU for each class, excluding missing classes."""
    dsc_scores = []
    iou_scores = []

    for class_id in range(num_classes):
        pred_mask = prediction == class_id
        gt_mask = ground_truth == class_id

        if not pred_mask.any() and not gt_mask.any():
            dsc_scores.append(None)
            iou_scores.append(None)
            continue

        intersection = np.logical_and(pred_mask, gt_mask).sum()
        pred_sum = pred_mask.sum()
        gt_sum = gt_mask.sum()

        dsc = (2.0 * intersection) / (pred_sum + gt_sum + 1e-7)
        dsc_scores.append(dsc)

        union = np.logical_or(pred_mask, gt_mask).sum()
        iou = intersection / (union + 1e-7)
        iou_scores.append(iou)

    return dsc_scores, iou_scores


def load_model(model_name, config):
    """Load the trained UNet model."""
    device = config["device"]
    model_path = f"best_{model_name}.pth"
    model = select_model(model_name, config)
    if device == "cuda":
        model.load_state_dict(torch.load(model_path))
    else:
        model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model = model.to(device)
    model.eval()
    return model

def process_mask(mask, target_size):
    """Resize and clip mask values."""
    transform = Compose([Resize(target_size, target_size)])
    mask = transform(image=mask)['image']
    mask = mask.astype(np.int64)
    mask = np.clip(mask, 0, 2)
    return mask

def process_batch(image_batch, mask_batch, model, config, output_dir="results1"):
    """Process a batch of images and masks."""
    os.makedirs(output_dir, exist_ok=True)
    device = config["device"]
    image_size = config["image_size"]

    all_dsc_scores = []
    all_iou_scores = []

    model.eval()
    with torch.no_grad():
        for idx, (images, masks) in enumerate(zip(image_batch, mask_batch)):
            images = images.to(device)
            masks = masks.to(device).long().squeeze(1)
            print(f"Shape of images: {images.shape}")
            print(f"Shape of masks: {masks.shape}")
            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1).cpu().numpy()

            for i in range(images.size(0)):
                prediction = predictions[i]
                ground_truth = masks[i]

                ground_truth = process_mask(ground_truth, image_size)

                dsc_scores, iou_scores = calculate_metrics(prediction, ground_truth)
                all_dsc_scores.append(dsc_scores)
                all_iou_scores.append(iou_scores)

                colors = np.array([[0, 0, 0], [0, 255, 0], [255, 0, 0]])
                prediction_rgb = colors[prediction]
                ground_truth_rgb = colors[ground_truth]

                plt.figure(figsize=(20, 8))
                plt.subplot(131)
                plt.imshow(images[i].cpu().squeeze(), cmap='gray')
                plt.title('Original Image')
                plt.axis('off')

                plt.subplot(132)
                plt.imshow(prediction_rgb)
                plt.title('Prediction')
                plt.axis('off')

                plt.subplot(133)
                plt.imshow(ground_truth_rgb)
                plt.title('Ground Truth')
                plt.axis('off')

                save_path = os.path.join(output_dir, f'result_{idx}_{i}.png')
                plt.savefig(save_path, bbox_inches='tight', dpi=300)
                plt.close()
                print(f"Saved result to {save_path}")

    return all_dsc_scores, all_iou_scores

def identify_classes(mask, num_classes=3):
    """Identify which classes are present in the mask."""
    present_classes = set(np.unique(mask))
    return {class_id: (class_id in present_classes) for class_id in range(num_classes)}



# Load Model
model_name = "SwinUNETR"
model = load_model(model_name, config)


# Track metrics for different class combinations
class_metrics = {
    "background_kidney": {"dsc": [], "iou": [], "count": 0},
    "background_kidney_tumor": {"dsc": [], "iou": [], "count": 0},
}

# Loop through test_loader
for batch_idx, (image_batch, mask_batch) in enumerate(test_loader):
    images = image_batch.to(config["device"])
    masks = mask_batch.to(config["device"]).long().squeeze(1)

    with torch.no_grad():
        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1).cpu().numpy()

        for i in range(images.size(0)):
            prediction = predictions[i]
            ground_truth = masks[i].cpu().numpy()

            # Identify classes present in the ground truth
            gt_classes = identify_classes(ground_truth, config["num_classes"])
            pred_classes = identify_classes(prediction, config["num_classes"])

            # Determine the category of the current image
            if gt_classes[1] and not gt_classes[2]:  # Background + Kidney
                category = "background_kidney"
            elif gt_classes[1] and gt_classes[2]:  # Background + Kidney + Tumor
                category = "background_kidney_tumor"
            else:
                continue  # Ignore images with only the background

            # Process and resize ground truth
            ground_truth = process_mask(ground_truth, config["image_size"])

            # Calculate metrics
            dsc_scores, iou_scores = calculate_metrics(prediction, ground_truth, config["num_classes"])

            # Update category metrics
            class_metrics[category]["dsc"].append(dsc_scores)
            class_metrics[category]["iou"].append(iou_scores)
            class_metrics[category]["count"] += 1


# Calculate and display metrics
print("\nMetrics by Category:")
for category, metrics in class_metrics.items():
    if metrics["count"] > 0:
        mean_dsc = [
            np.nanmean([scores[i] for scores in metrics["dsc"] if scores[i] is not None])
            for i in range(config["num_classes"])
        ]
        mean_iou = [
            np.nanmean([scores[i] for scores in metrics["iou"] if scores[i] is not None])
            for i in range(config["num_classes"])
        ]

        print(f"\nCategory: {category}")
        print(f"Number of Images: {metrics['count']}")

        for i, class_name in enumerate(['Background', 'Kidney', 'Tumor']):
            print(f"  {class_name}:")
            print(f"    Mean DSC: {mean_dsc[i]:.3f}")
            print(f"    Mean IoU: {mean_iou[i]:.3f}")




Metrics by Category:

Category: background_kidney
Number of Images: 1620
  Background:
    Mean DSC: 0.999
    Mean IoU: 0.999
  Kidney:
    Mean DSC: 0.943
    Mean IoU: 0.911
  Tumor:
    Mean DSC: 0.000
    Mean IoU: 0.000

Category: background_kidney_tumor
Number of Images: 753
  Background:
    Mean DSC: 0.999
    Mean IoU: 0.998
  Kidney:
    Mean DSC: 0.920
    Mean IoU: 0.865
  Tumor:
    Mean DSC: 0.622
    Mean IoU: 0.553


In [14]:
# Process Test Data
all_dsc_scores = []
all_iou_scores = []

# Loop through test_loader
for batch_idx, (image_batch, mask_batch) in enumerate(test_loader):
    # Unpack single image and mask from the batch
    images = image_batch.to(config["device"])
    masks = mask_batch.to(config["device"]).long().squeeze(1)

    with torch.no_grad():
        # Get model predictions
        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1).cpu().numpy()

        for i in range(images.size(0)):
            prediction = predictions[i]
            ground_truth = masks[i].cpu().numpy()

            # Process and resize ground truth mask
            ground_truth = process_mask(ground_truth, config["image_size"])

            # Calculate DSC and IoU
            dsc_scores, iou_scores = calculate_metrics(prediction, ground_truth, config["num_classes"])
            all_dsc_scores.append(dsc_scores)
            all_iou_scores.append(iou_scores)

            # Visualization (optional)
            if batch_idx % 10 == 0:  # Save every 10th image
                colors = np.array([[0, 0, 0], [0, 255, 0], [255, 0, 0]])  # Class colors
                prediction_rgb = colors[prediction]
                ground_truth_rgb = colors[ground_truth]

                plt.figure(figsize=(20, 8))
                plt.subplot(131)
                plt.imshow(images[i].cpu().squeeze(), cmap='gray')
                plt.title('Original Image')
                plt.axis('off')

                plt.subplot(132)
                plt.imshow(prediction_rgb)
                plt.title('Prediction')
                plt.axis('off')

                plt.subplot(133)
                plt.imshow(ground_truth_rgb)
                plt.title('Ground Truth')
                plt.axis('off')

                output_dir = "results1"
                os.makedirs(output_dir, exist_ok=True)
                save_path = os.path.join(output_dir, f'result_{batch_idx}_{i}.png')
                plt.savefig(save_path, bbox_inches='tight', dpi=300)
                plt.close()
                print(f"Saved result to {save_path}")

# Aggregate Metrics
if all_dsc_scores:
    mean_dsc_per_class = [
        np.nanmean([scores[i] for scores in all_dsc_scores if scores[i] is not None])
        for i in range(config["num_classes"])
    ]
    mean_iou_per_class = [
        np.nanmean([scores[i] for scores in all_iou_scores if scores[i] is not None])
        for i in range(config["num_classes"])
    ]

    class_names = ['Background', 'Kidney', 'Tumor']
    print("\nOverall Metrics:")
    for i, class_name in enumerate(class_names):
        print(f"{class_name}:")
        print(f"Mean DSC: {mean_dsc_per_class[i]:.3f}")
        print(f"Mean IoU: {mean_iou_per_class[i]:.3f}")

    print("\nOverall Mean Metrics:")
    print(f"Mean DSC: {np.nanmean(mean_dsc_per_class):.3f}")
    print(f"Mean IoU: {np.nanmean(mean_iou_per_class):.3f}")

Saved result to results1/result_0_0.png
Saved result to results1/result_10_0.png
Saved result to results1/result_20_0.png
Saved result to results1/result_30_0.png
Saved result to results1/result_40_0.png
Saved result to results1/result_50_0.png
Saved result to results1/result_60_0.png
Saved result to results1/result_70_0.png
Saved result to results1/result_80_0.png
Saved result to results1/result_90_0.png
Saved result to results1/result_100_0.png
Saved result to results1/result_110_0.png
Saved result to results1/result_120_0.png
Saved result to results1/result_130_0.png
Saved result to results1/result_140_0.png
Saved result to results1/result_150_0.png
Saved result to results1/result_160_0.png
Saved result to results1/result_170_0.png
Saved result to results1/result_180_0.png
Saved result to results1/result_190_0.png
Saved result to results1/result_200_0.png
Saved result to results1/result_210_0.png
Saved result to results1/result_220_0.png
Saved result to results1/result_230_0.png
Sav